In [9]:
import math
import numpy as np
from scipy.integrate import odeint
import pandas as pd
N=1000
kp = [1,1,1,1]
n_p = len(kp)
PS = []
comp_PS = []
n_base = math.ceil(N/n_p)
y0 = [30,4]
LH_data = pd.read_csv('Lynx_hare_data.csv')
data=LH_data.as_matrix(columns=[LH_data.columns])
tspan=data[:,0]#np.linspace(0,20,21)    
data_H=data[:,1]
data_L=data[:,2]
def Sobol_obj(kp):
#a function that works with the trial Sobol Method program
# the trial function is fx: x1+5*x2*x3^2; 
    def model(y, t):
         y1=y[0];
         y2=y[1];
         dy1=kp[0]*y1-kp[1]*y1*y2
         dy2=-kp[2]*y2+kp[3]*y1*y2
         dy=[dy1,dy2] 
         return dy
    soln = odeint(model, y0, tspan)
    
    return np.sum(soln[:,0]-data_H)**2 + np.sum(soln[:,1]-data_L)**2

a = np.ceil((10*np.random.random((int(n_base),1))))
b = np.ceil((10*np.random.random((int(n_base),1))))
c = np.ceil((10*np.random.random((int(n_base),1))))
d = np.ceil((10*np.random.random((int(n_base),1))))

#PS=[np.ceil((1000*np.random.random((int(n_base),1))) + 1), np.ceil((10 + 90*np.random.random((int(n_base),1)))), np.ceil((1+ 10*np.random.random((int(n_base),1))))]
#comp_PS = [np.ceil((1000*np.random.random((int(n_base),1))) + 1), np.ceil((10 + 90*np.random.random((int(n_base),1)))), np.ceil((1+ 10*np.random.random((int(n_base),1))))]
PS = np.column_stack((a,b,c,d))

a = np.ceil((10*np.random.random((int(n_base),1))))
b = np.ceil((10*np.random.random((int(n_base),1))))
c = np.ceil((10*np.random.random((int(n_base),1))))
d = np.ceil((10*np.random.random((int(n_base),1))))

comp_PS = np.column_stack((a,b,c,d))
output = [[0] for i in range(int(n_base))]
c_out_1 = [[0] * n_p for i in range(int(n_base))]
c_out_t = [[0] * n_p for i in range(int(n_base))]
for i in range(int(n_base)):
    #kp = [PS[i][0],PS[i][1],PS[i][2]]
    kp = PS[i][:]
    output[i] = (Sobol_obj(kp))
    for j in range(int(n_p)):
        kp = np.append(comp_PS[i][0:j if j>0 else 0], PS[i][j])
        kp = np.append(kp, comp_PS[i][j+1:n_p])
        c_out_1[i][j] = Sobol_obj(kp)
        
        kp = np.append(PS[i][0:j if j>0 else 0], comp_PS[i][j])
        kp = np.append(kp, PS[i][j+1:n_p])
        c_out_t[i][j] = Sobol_obj(kp)


f0 = 0
D = 0
for i in range(int(n_base)):
    f0 = f0+output[i]/n_base
    D = D+(output[i]**2)/n_base

D=D-f0**2
Dj= [[D] for i in range(int(n_p))]
Dtotj= [[0] for i in range(int(n_p))]
    
for i in range(int(n_base)):
    for j in range(int(n_p)):
        Dj[j] = Dj[j] - ((output[i]-c_out_1[i][j])**2)/(2*n_base)
        Dtotj[j] = Dtotj[j] + ((output[i]-c_out_t[i][j])**2)/(2*n_base)
Dmat = [[D] for i in range(int(n_p))]
Sob_1 = np.divide(Dj,Dmat)
Sob_t = np.divide(Dtotj,Dmat)
#Sob_1 = Dj/(ones())

C:\Users\GTOBAN\Anaconda2\lib\site-packages\scipy\integrate\odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [11]:
print "n_p = ", n_p
print ''
print "D = ", D
print ""
print "Dj"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(Dj)))
print ""
sens_ind = Sob_t
rank_sens = sorted(Sob_t)
rank_sens_p = sorted(range(len(Sob_t)), key=lambda x:Sob_t[x])
print "sens_ind"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(sens_ind)))
print ""
print "rank_sens"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(rank_sens)))
print ""
print "rank_sens_p"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(rank_sens_p)))
print ""
print "PS"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(PS)))
print ""
print "comp_PS"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(comp_PS)))
print ""
print "output"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(output)))
print ""
print "c_out_1"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(c_out_1)))
print ""
print "c_out_t"
print('\n'.join('{}: {}'.format(*k) for k in enumerate(c_out_t)))

n_p =  4

D =  6.75959280178e+291

Dj
0: [  1.36701000e+291]
1: [  3.36579588e+291]
2: [  3.09762127e+291]
3: [  3.36613330e+291]

sens_ind
0: [ 0.5020209]
1: [ 0.50207082]
2: [ 0.50201839]
3: [ 1.04917171]

rank_sens
0: [ 0.50201839]
1: [ 0.5020209]
2: [ 0.50207082]
3: [ 1.04917171]

rank_sens_p
0: 2
1: 0
2: 1
3: 3

PS
0: [  1.   3.   2.  10.]
1: [  7.   4.   9.  10.]
2: [ 10.  10.   5.   7.]
3: [ 1.  6.  3.  5.]
4: [ 10.  10.   2.   5.]
5: [ 9.  5.  2.  5.]
6: [ 4.  5.  7.  6.]
7: [  4.  10.   6.  10.]
8: [  9.   9.   5.  10.]
9: [  5.  10.   6.   2.]
10: [ 4.  4.  6.  1.]
11: [  7.  10.   6.   1.]
12: [ 10.   9.   9.   6.]
13: [ 4.  6.  7.  4.]
14: [ 10.   8.   8.   8.]
15: [  6.  10.   2.   4.]
16: [  1.  10.   6.   8.]
17: [ 1.  4.  8.  1.]
18: [ 4.  6.  4.  8.]
19: [  8.   3.  10.   4.]
20: [ 7.  8.  2.  2.]
21: [ 7.  2.  9.  1.]
22: [  2.  10.   2.   4.]
23: [  6.   1.  10.  10.]
24: [ 4.  2.  9.  6.]
25: [  5.  10.   7.   5.]
26: [ 8.  9.  2.  5.]
27: [ 2.  3.  1.  5.]
28: [ 1.